In [1]:
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as mplt
import scipy.integrate as scyint

import pyaccel
import pymodels
from mathphys import constants
from mathphys.beam_optics import beam_rigidity
import scipy.special as _special

In [2]:
acc = pymodels.si.create_accelerator()
acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(acc)
spos=pyaccel.lattice.find_spos(acc, indices='closed')

# rf acceptance
cav = pyaccel.lattice.find_indices(acc, 'fam_name', 'SRFCav')[0]
acc[cav].voltage = 1.5e6
acc.cavity_on = True
acc.radiation_on = True
acc.vchamber_on = True

eqparams = pyaccel.optics.EqParamsFromBeamEnvelope(acc)
envelopes = eqparams.envelopes

ltime=pyaccel.lifetime.Lifetime(acc)
accep = pyaccel.optics.calc_touschek_energy_acceptance(acc, track= True, check_tune=True)

# ltime.equi_params.rf_acceptance

### Function to see the lattice and beta function

In [4]:
def see_lattice(acc, pos):
    spos = pyaccel.lattice.find_spos(acc, indices='closed')
    twi_l,*_ = pyaccel.optics.calc_twiss(acc, indices='closed')
    betax = twi_l.betax
    position = pos

    element_idex = np.argmin(np.abs(spos-pos))
    
    fig, ax = mplt.subplots()
    ax.plot(spos, np.sqrt(betax) , color='lightgrey')
    pyaccel.graphics.draw_lattice(acc, offset=0, height= 0.3, gca = True)

    mplt.tight_layout()
    ax.plot(spos[element_idex], 0, 'ko')

    mplt.show()

### slightly modified anallytical Piwinski's density probability 

In [5]:
def f_function_arg_mod(kappa, kappam, b1_, b2_):
    tau = (np.tan(kappa)**2)[:, None]
    taum = (np.tan(kappam)**2)
    beta = beam_rigidity(energy=3)[2]
    ratio = tau/taum/(1+tau)
    arg = (2*tau+1)**2 * (ratio - 1)/tau
    arg += tau - np.sqrt(tau*taum*(1+tau))
    arg -= (2+1/(2*tau))*np.log(ratio)
    arg *= np.sqrt(1+tau)
#     arg *= beta* np.cos(kappa)[:, None]**2
    arg *= 1/(2*np.sqrt(tau)) * 1/(1+tau)
    arg *= 2* beta* np.sqrt(tau)

    bessel = np.exp(-(b1_-b2_)*tau)*_special.i0e(b2_*tau)
    return arg * bessel

def f_integral_simps_l_mod(taum, b1_, b2_):
    kappam = np.arctan(np.sqrt(taum))
    npts = int(9*1000)
    dkappa = (np.pi/2-kappam)/npts
    kappa = np.linspace(kappam, np.pi/2, npts+1)
    func = f_function_arg_mod(kappa, kappam, b1_, b2_)

    # Simpson's 3/8 Rule - N must be mod(N, 3) = 0
    val1 = func[0:-1:3, :] + func[3::3, :]
    val2 = func[1::3, :] + func[2::3, :]
    f_int = 3*dkappa/8*np.sum(val1 + 3*val2, axis=0)

    # # Simpson's 1/3 Rule - N must be mod(N, 2) = 0
    # val1 = func[0::2, :] + func[2::2, :]
    # val2 = func[1::2, :]
    # f_int = dkappa/3*np.sum(val1+4*val2, axis=0)
    f_int *= 2*np.sqrt(np.pi*(b1_**2-b2_**2))*taum
    return f_int


# function returns the desired element index of dipoles, quadrupoles, sextupoles and any other 
# element that is desired, the only thing that is needed is to pass a string 

def el_idx_collector(acc, lname):
    all_index = []
    fam_data = pymodels.si.get_family_data(acc)

    for string in lname:
        element_index = []
        array_idx = np.array(fam_data[string]['index'], dtype=object)

        for k, lista in enumerate(array_idx):
            length = len(lista)

            if length % 2 != 0:
                ind = int((length-1)/2)
                element_index.append(lista[ind])

            else:
                try:
                    ind = int(length/2 + 1)
                    element_index.append(lista[ind])
                
                except IndexError:
                    ind = int(length/2 - 1)
                    element_index.append(lista[ind])
                    
        all_index.append(element_index)
    
    return all_index

def select_idx(list_, param1, param2):
    arr = np.array(list_)
    
    n_arr = arr[param1:param2+1]
    
    return n_arr

### Standard deviation calculated to compare anallytical and monte-calor simulation

In [6]:
# Functions for calculating the standard deviation (for analytical and Monte-carlo simulation)

def calc_smc(f_densp, deltp, f_densn, deltn):
    secnd_mps = []
    secnd_mns = []
    for k, iten in enumerate(f_densp):
        
        pmp = scyint.trapz(deltp[k] * iten, deltp[k])
        smp = scyint.trapz((deltp[k])**2 * iten, deltp[k])

        pmn = scyint.trapz(deltn[k] * f_densn[k], deltn[k])
        smn = scyint.trapz((deltn[k])**2 * f_densn[k], deltn[k])
        
        secnd_mps.append(smp - pmp**2)
        secnd_mns.append(smn - pmn**2)
        
    secnd_mps = np.array(secnd_mps)
    secnd_mns = np.array(secnd_mns)
    
    return secnd_mps, secnd_mns

def calc_hist(hp, hn):
    secnd_mps = []
    secnd_mns = []
    for k, iten in enumerate(hp):
        
        secnd_mps.append(np.std(iten)**2)
        secnd_mns.append(np.std(hn[k])**2)

    secnd_mps = np.array(secnd_mps)
    secnd_mns = np.array(secnd_mns)
    
    return secnd_mps, secnd_mns

# caculating the normalized probability density 

In [7]:
def get_dis(lsps, npt, accep, norm):
    model = pymodels.si.create_accelerator()
    model = pymodels.si.fitted_models.vertical_dispersion_and_coupling(model)
    
    spos = pyaccel.lattice.find_spos(model, indices='closed')
    npoints = int((spos[-1]-spos[0])/0.1)
    scalc= np.linspace(spos[0],spos[-1], npoints)
    
    daccp = np.interp(scalc, spos, accep[1])
    daccn = np.interp(scalc, spos, accep[0])

    beta = beam_rigidity(energy=3)[2]
    taum_p = (beta*daccp)**2
    taum_n = (beta*daccn)**2

    kappam_p = np.arctan(np.sqrt(taum_p))
    kappam_n = np.arctan(np.sqrt(taum_n))
    
    ltime = pyaccel.lifetime.Lifetime(model)
    b1, b2 = ltime.touschek_data['touschek_coeffs']['b1'],ltime.touschek_data['touschek_coeffs']['b2']

    calc_dp, calc_dn = [], []
    deltasp, deltasn = [], []
    indices, indices_model= [], []

    for indx, s in enumerate(lsps):
        
        dif = np.abs(scalc - s)
        idx = np.argmin(dif)
        indices.append(idx)
        
        dif1 = np.abs(spos - s)
        idx_model = np.argmin(dif1)

#         kappam_p0 = kappam_p[idx]
#         kappam_n0 = kappam_n[idx]
        
        kappam_p0 = 0.001 # teste sugerido pelo ximenes
        kappam_n0 = 0.001

        
        if norm:
            kappap = np.linspace(kappam_p0, np.pi/2, npt)
            deltap = 1/beta * np.tan(kappap)
            kappan = np.linspace(kappam_n0, np.pi/2, npt)
            deltan = 1/beta * np.tan(kappan)

            y_p = f_function_arg_mod(kappa=kappap,kappam=kappam_p0,b1_=b1[idx],b2_=b2[idx])
            y_n = f_function_arg_mod(kappa=kappan,kappam=kappam_n0,b1_=b1[idx],b2_=b2[idx])
            y_p = y_p.squeeze()
            y_n = y_n.squeeze()

            norm_facp = scyint.trapz(y_p, deltap)
            norm_facn = scyint.trapz(y_n, deltan)

    #         Normalizing the probability density function

            y_p /= (norm_facp)
            y_n /= (norm_facn)

            calc_dp.append(y_p)
            calc_dn.append(y_n)
            deltasp.append(deltap)
            deltasn.append(deltan)
            indices_model.append(idx_model)
            
        
    calc_dp = np.array(calc_dp)
    calc_dn = np.array(calc_dn)
    deltasp = np.array(deltasp)
    deltasn = np.array(deltasn)
    indices = np.array(indices)
    indices_model = np.array(indices_model)


    return calc_dp, calc_dn, deltasp, deltasn, indices, indices_model

In [28]:
# secnd_mps = []
# secnd_mns = []


def plot_hdis(acc, l_index):
    
    fig, axis = mplt.subplots(1,l_index.squeeze().size, figsize=(10,6))
    mplt.suptitle('Comparação entre densidade de probabilidade analítica e de M.C.', fontsize=15)
    spos = pyaccel.lattice.find_spos(acc, indices='closed')
    nb = int((spos[-1] - spos[0])/0.1)
    scalc = np.linspace(spos[0],spos[-1], nb)

    for c,iten in enumerate(l_index):
        ax = axis[c]
        
        if c == 0:
            ax.set_ylabel(r'PDF normalizada', fontsize=15)

        idx = np.argmin(np.abs(spos[iten] - scalc))

        ax.set_title('{}'.format(acc[iten].fam_name))
        ax.plot(deltp[idx], f_densp[idx], color='blue', label='Analytic ({:.2f} [m])'.format(scalc[idx]))
        
        ax.tick_params(axis='both', labelsize=14)

        ax.hist(hn[idx], density=True, bins=200, color='lightgrey', label='Monte-Carlo')
        ax.plot(-deltn[idx], f_densn[idx], color='blue')
    #     ax.set_yscale('log')

        ax.hist(hp[idx],density=True, bins=200,color='lightgrey')
    #     ax.set_ylim(1e-1,1e3)
        ax.set_xlim(-0.3,0.3)

        ax.set_xlabel(r'$\delta$ [%]', fontsize=15)
        ax.legend()

        ax.grid(axis='y', ls='--', alpha=0.5)
    #     fig.tight_layout()
        mplt.show
    
    return idx


# Isso é legal, mas é interessante obter esse desvio padrão para todos os pontos do anel para verificar se 
# existe alguma correlação com as funções ópticas

In [ ]:
fam = pymodels.si.get_family_data(acc)

In [26]:
lname = ['BC','B1','B2','Q1','Q2','Q3','Q4','SDA1', 'QFA', 'QFB','SFP2', 'girder']

all_index = el_idx_collector(acc, lname)

BC_index = np.array(all_index[0], dtype=object)
B2_index = all_index[2]
B1_index = all_index[1]

SDA1_index = all_index[-4]
QFA = all_index[-3]
QFB = all_index[-2]

girder = all_index[-1]

In [33]:
lindex = select_idx(girder,0,1)
lindex

array([17, 44])

In [34]:
plot_hdis(acc, lindex)

55

# Transformation matrix from $(j,k,l)$ to $(x,y,z)$

- To perfome a transformation on the reference frame it is necessary to obtain the transformation matrix 

$$ \begin{pmatrix} \cos (\chi) \\ 0 \\ \sin (\chi) \end{pmatrix}_{j,k,l} \overset{M}\Rightarrow \frac{1}{p}\begin{pmatrix} p_{1x} \\ p_{1y} \\ p_{1z} \end{pmatrix}_{x,y,z} $$ 

In this case, $\frac{p_{1z}}{p} = \sqrt{(1+\delta)^{2}-{x^{'}}^2-{y^{'}}^2}$

# Touschek scattering simulation

In [15]:
def create_particles(cov_matrix, num_part):
    
    # permute indices to change the order of the columns:
    # [rx, px, ry, py, de, dl]^T -> [px, py, de, rx, ry, dl]^T
    idcs = [1, 3, 4, 0, 2, 5]
    # [px, py, de, rx, ry, dl]^T -> [rx, px, ry, py, de, dl]^T
    idcs_r = [3, 0, 4, 1, 2, 5]
    
    cov_matrix = cov_matrix[:, idcs][idcs, :]
    
    sig_xx = cov_matrix[:3, :3]
    sig_xy = cov_matrix[:3, 3:]
    sig_yx = cov_matrix[3:, :3]
    sig_yy = cov_matrix[3:, 3:]
    inv_yy = np.linalg.inv(sig_yy)
    
    part1 = np.random.multivariate_normal(np.zeros(6), cov_matrix, num_part).T
    # changing the matrices' columns order 
    part2 = part1.copy()

    vec_a = part2[3:]
    new_mean = (sig_xy @ inv_yy) @ vec_a
    new_cov = sig_xx - sig_xy @ inv_yy @ sig_yx

    part2[:3] = np.random.multivariate_normal(np.zeros(3), new_cov, num_part).T
    part2[:3] += new_mean
        
    part2 = part2[idcs_r, :]
    part1 = part1[idcs_r, :]

    return part1, part2


def get_cross_section_distribution(psim, npts=3000):
    beta_bar = 0
    psi = np.logspace(np.log10(np.pi/2 - psim), 0, npts)
    psi = np.pi/2 - psi
    psi = psi[::-1] # this step is necessary to reverse the order of psi
    cpsi = np.cos(psi)
    cross = np.zeros(cpsi.size)
    if beta_bar > 1e-19: # I think it maybe will be related to the relativistic regime
        cross += (1 + 1/beta_bar**2)**2 * (2*(1 + cpsi**2)/cpsi**3 - 3/cpsi)
    cross += 4/cpsi + 1
    cross *= np.sin(psi)
    cross = scyint.cumtrapz(cross, x=psi, initial=0.0)
    cross /= cross[-1]
    return psi, cross


def cross_section_draw_samples(psim, num_part):
    psi, cross = get_cross_section_distribution(psim)
    crs = np.random.rand(num_part)
    return np.interp(crs, cross, psi)


def scatter_particles(part1, part2, de_min):
    gamma = 3e9 / 0.510e6
    beta = np.sqrt(1 - 1/gamma/gamma)
    num_part = part1.shape[1]

    xl1, yl1, de1 = part1[1], part1[3], part1[4]
    xl2, yl2, de2 = part2[1], part2[3], part2[4]

    # calculating the changing base matrix for every particle
    pz1 = np.sqrt((1+de1)**2 - xl1*xl1 - yl1*yl1)
    pz2 = np.sqrt((1+de2)**2 - xl2*xl2 - yl2*yl2)
        
    # desired vectors to construct the transformation matrix
    p_1 = np.vstack([xl1, yl1, pz1])
    p_2 = np.vstack([xl2, yl2, pz2])
        
    # new coordinate system
    p_j = p_1 + p_2
    p_j /= np.linalg.norm(p_j, axis=0) # sum
    p_k = np.cross(p_1.T, p_2.T).T
    p_k /= np.linalg.norm(p_k, axis=0) # cross product
    p_l = np.cross(p_j.T, p_k.T).T

    jkl2xyz = np.zeros([num_part, 3, 3])
    jkl2xyz[:, :, 0] = p_j.T
    jkl2xyz[:, :, 1] = p_k.T
    jkl2xyz[:, :, 2] = p_l.T
    
    # calculating theta and zeta
    theta = xl1 - xl2 
    zeta = yl1 - yl2
    # defining the value of the scattering angle chi
    chi = np.sqrt(zeta**2 + theta**2)/2

    # draw the scattering angles from uniform distribution:
    phi = np.random.rand(num_part)* 2*np.pi
    psi = np.random.rand(num_part)* np.pi/2
    
    # draw the psi angle from the cross section probability density:
    # we need to define a maximum angle to normalize the cross section
    # distribution because it diverges when the full interval [0, pi/2]
    # is considered.
    # To estimate this angle we define a threshold for the minimum energy
    # deviation we care about (de_min) and consider the worst case scenario
    # in terms of chi that could create such scattering. 
    # The worst case happens when chi is maximum, because in this way a small
    # scattering angle would create a larger energy deviation.
    # We considered here a chi twice as large as the maximum chi draw from
    # the particles distribution.
    # This method of doing things should be tested and thought about very
    # carefully, though.
    psim = np.arccos(de_min/gamma/(chi.max()*2))
    fact=psim*2/np.pi
    print(psim*2/np.pi)
    psi = cross_section_draw_samples(psim, num_part)
    
    # new momentum in j,k,l (eq. 16 of Piwinski paper)
    gammat = gamma/np.sqrt(1 + beta*beta*gamma*gamma*chi*chi)
    dp_ = np.sin(chi[None, :]) * np.vstack([
        gammat*np.cos(psi),
        np.sin(psi)*np.cos(phi),
        np.sin(psi)*np.sin(phi)])
    p_prime1 = np.zeros((3, chi.size))
    p_prime1[0] = np.cos(chi)
    p_prime2 = p_prime1.copy()
    p_prime1 += dp_
    p_prime2 -= dp_

    # returning to momentum x,y,z
    pnew_1 = (jkl2xyz @ p_prime1.T[:, :, None]).squeeze().T
    pnew_2 = (jkl2xyz @ p_prime2.T[:, :, None]).squeeze().T

    delta1 = np.linalg.norm(pnew_1, axis=0) - 1
    delta2 = np.linalg.norm(pnew_2, axis=0) - 1
        
    part1_new = part1.copy()
    part1_new[1] = pnew_1[0]
    part1_new[3] = pnew_1[1]
    part1_new[4] = delta1
            
    part2_new = part2.copy()
    part2_new[1] = pnew_2[0]
    part2_new[3] = pnew_2[1]
    part2_new[4] = delta2

    return part1_new, part2_new, fact

In [ ]:
pos = 64.5
idx = np.argmin(np.abs(scalc-pos))
b1[idx]

In [741]:
np.argmin(np.abs(scalc-pos)), np.min(np.abs(scalc-pos)), scalc[643]

(643, 0.04656343110673333, 64.32356343110673)

In [941]:
mplt.figure()
mplt.plot(scalc, b1)
mplt.scatter(scalc[idx], b1[idx], color='k')
pyaccel.graphics.draw_lattice(acc, offset=0, height=5000, gca=True)
mplt.show()

In [10]:
def histgms(l_spos,num_part,accep, de_min):
    spos=pyaccel.lattice.find_spos(acc, indices='closed')
    npoint = int((spos[-1]-spos[0])/0.1)
    scalc = np.linspace(spos[0], spos[-1], npoint)
    histsp1=[]
    histsp2=[]
    indices=[]
    for iten in l_spos:
        
        idx_model = np.argmin(np.abs(spos - iten))
        idx = np.argmin(np.abs(scalc - iten))

        env = envelopes[idx_model]
        part1, part2= create_particles(env, num_part)
        part1_new, part2_new, fact = scatter_particles(part1, part2, de_min)

#         acpp, acpn = accep[1][idx], accep[0][idx]
        
#         check1 = acpp - part1_new[4]
#         check2 = -(acpn - part2_new[4])
#         cond2 = check2<0
#         cond1 = check1<0
        
#         ind1 = np.where(cond1)[0] # apenas os desvios de energia maiores que a a aceitancia da máquina
#         ind2 = np.where(cond2)[0]
        
        ind1 = np.where(part1_new[4]>=0.001) # teste sugerido pelo ximenes
        ind2 = np.where(part2_new[4]<=-0.001)
        
        histsp1.append((part1_new[4][ind1]))
        histsp2.append((part2_new[4][ind2]))
        indices.append(idx)
        
    hist1=np.array(histsp1, dtype='object')
    hist2=np.array(histsp2, dtype='object')
    indices=np.array(indices)
    
    return histsp1, histsp2, indices
    
def plotdef(mp, mn, indices, des):
    num=mp.shape[1]
    l=mp.shape[0]
    fig, axs = mplt.subplots(l, num, sharey=True, sharex=True)
    fig.suptitle(r'Gráficos n_count $\times$ $\delta$')
#     axs=np.array(axs, dtype='object')
    
    for ind, tens in enumerate(des):
        for indx, iten in enumerate(indices):
            
            ax=axs[ind][indx]
            ax.hist(mp[ind][indx], bins=200, density=True, label=r'$\delta_m=${:.3%}'.format(tens), color='black')
            ax.hist(mn[ind][indx], bins=200, density=True,  color='blue')
#             ax.set_yticks([0,25,50,75,100,125])
            ax.set_xticks([-0.2,-0.1,0,0.1,0.2])
            ax.legend()
#             ax.set_xlabel('$\delta$')
            if ind==0:
                ax.set_title('{}, s = {:.2f} [m]'.format(acc[iten].fam_name,spos[iten]))
            else:
                pass
    
    mplt.show()

In [11]:
# des = np.array([1e-2,1e-3,1e-4,1e-5])

# mp=[]
# mn=[]
# for idx, iten in enumerate(des):
#     hist=histgms(l_spos=lspos,num_part=50000, accep=accep,de_min=iten)
#     histp, histn, indices = hist
#     mp.append(histp)
#     mn.append(histn)
    
# mp = np.array(mp, dtype='object')
# mn = np.array(mn, dtype='object')

In [12]:
# plotdef(mp, mn, indices, des)

# Comparasion between analytically calculated probability and monte carlo simulation 

- this cel bellow is necessary when I was testing the difference between the monte carlo simulation and the analytical probability density

In [9]:
# Lista com os dipolos B1 

lspos = np.array([5.3, 31.38, 57.35])

# Lista com os dipolos BC

lspos = np.array([13, 38.79, 64.78])

# Lista com os dipolos B2

lspos = np.array([9.1, 34.91, 61.11])

# Lista com quadrupolos 

lspos = np.array([4.050, 29.212, 55.129])

# Lista com sextupolos

lspos = np.array([3.727, 29.529, 55.471])

# Lista com trechos retos

lspos = np.array([26, 51.85, 103.54])


In [16]:
# lspos = np.array([scalc[BC_index[1]]])
spos = pyaccel.lattice.find_spos(acc, indices='closed')
nb = int((spos[-1]-spos[0])/0.1)
scalc = np.linspace(spos[0], spos[-1], nb)

hp,hn,ind = histgms(l_spos=scalc, num_part=5000, accep=accep, de_min=1e-4)
# ind, spos[ind]

0.9992573708988877
0.9992584099061698
0.999328901670686
0.9993565928803583
0.9992397974282787
0.9993332586125413
0.9993362175408294
0.9993374440600716
0.9992752188600968
0.9993612848649879
0.9991648474113719
0.9992113515913666
0.9992268204913566
0.9991520623926118
0.9991229406816019
0.9991906219803298
0.9992642122922856
0.9992045562153866
0.9991703244579265
0.999311925156826
0.9992070202342292
0.9992193353061878
0.9992219429491935
0.9992549627393168
0.9991799989646123
0.9991727509161857
0.9991807302414766
0.9992991898515208
0.999182641048606
0.999192502928646
0.9992186508846149
0.9992914646468288
0.9992890753863236
0.9992345120682439
0.9992705538486529
0.9992564561013842
0.9993171848889015
0.9993239101989629
0.9992134578058545
0.9992044721444725
0.9992114965573671
0.9992735389880967
0.9993428488541828
0.9994019685885368
0.9994039317580301
0.9994523133687011
0.999563992489027
0.9996473597231588
0.9997091249554128
0.9996985042085466
0.9997363143801543
0.9997291238392872
0.999831531031707

0.9996779822603167
0.9996686889615719
0.9997351135122015
0.9996538984684222
0.9996597733993244
0.9996700234212522
0.9997505297019189
0.9997720185760657
0.9997625216286314
0.9997764357562875
0.9997797681162767
0.9998067873750255
0.9997664001294416
0.999776316735848
0.9998255637341023
0.999823265530222
0.9998132868063876
0.9997833094217762
0.9997873068630111
0.9998207457947406
0.9998245912047404
0.999848580316225
0.9998115876298214
0.9998322483310397
0.9998052621718317
0.9998393544348566
0.9998225363386408
0.9998263458483776
0.999820165732481
0.9997963610748225
0.9997986035622677
0.9997775957022906
0.9997767024118482
0.9998070408535321
0.9997545929706005
0.9997729310439218
0.9997480496699966
0.9997437313324214
0.9997291620759867
0.9997440158394021
0.9997034585958371
0.9996869501360075
0.9996994035894086
0.9997121097104515
0.9997022226994328
0.9996748096750684
0.9997021737467243
0.9995318591823211
0.9995924993768371
0.9995772839495591
0.9995553953290373
0.9995832693328085
0.99963333010842

0.999564248372
0.9995550740947732
0.9995527180090178
0.9996314262448038
0.99963731150234
0.999579388257229
0.9996640573005329
0.9996650063782201
0.9997748110501019
0.9996775134670943
0.9997769290979454
0.9997882929313989
0.999798435517148
0.9998274683773543
0.9998761995024454
0.9998692895756738
0.999914406200209
0.9999021151199392
0.9999136320178706
0.999913800028273
0.9999075966087275
0.999889486615675
0.9998206559003577
0.9998333066162539
0.9997575027198642
0.9997940832192751
0.9997293111843748
0.9997373586737522
0.9996896449870482
0.9996145075458978
0.9995917695546012
0.9996124724863502
0.9995597910427673
0.9995813847297679
0.9995400774706764
0.9995457884400787
0.9995207018780392
0.9995300523360169
0.9995356381685871
0.9995821635148223
0.9995645869479355
0.9995905748462106
0.9996182979763433
0.9996707632862254
0.9996727039674131
0.9997276707514609
0.999724577005617
0.9997540263733807
0.9998575254097345
0.9998725123287053
0.9998562091754358
0.9998755007968678
0.9999113964714003
0.999

0.9996179747201557
0.9996675514896571
0.9996378938671288
0.9996558226840749
0.9996404904288205
0.999630229098114
0.9996259071418484
0.9996359045706211
0.9996215792827161
0.9997172605532364
0.9996933466311525
0.9997214927773531
0.999802115704792
0.9997835912764786
0.9997891397398738
0.9998066505897533
0.9998159909267551
0.9998320015143309
0.9998478204970415
0.9998777132591877
0.9998992369031944
0.9999059495350446
0.9999178598064286
0.9999092991673771
0.9999029219072835
0.9998993131411734
0.9998697020049637
0.999860787655386
0.9998393695212252
0.9998366966013748
0.9997588505169921
0.9997597807855015
0.9997237765215707
0.9997371574550372
0.9996455599682486
0.9996737878344135
0.99967349632651
0.999656227824089
0.9996519164472589
0.9996519593661207
0.9996574199828203
0.9996793245483974
0.9996904623308948
0.9996640022990027
0.999689358209763
0.9996792816060036
0.9997389889360279
0.9997656372487764
0.9997812387002217
0.9997769812798045
0.9998443761981505
0.9998276862191442
0.9998686472928434


0.9998223088304002
0.9998278027592056
0.9998391385883669
0.9998253773834455
0.9998200108703725
0.9998339314735023
0.9998299012059737
0.9998731922934794
0.9998633688965899
0.9998510517778356
0.9997688541373888
0.9997771375679653
0.9998010385166706
0.9997724461266179
0.9997954516216734
0.9997867494148115
0.9997855831346061
0.9997459839295023
0.9997724422758705
0.9997024560653526
0.9997273303572356
0.9997263418282115
0.9997301918736147
0.9997246712508666
0.9997545378327088
0.9997294952952273
0.9996362162089866
0.9996262549359105
0.999648001391546
0.9996027649948683
0.9995587551653786
0.999657031726677
0.9995576118033863
0.999562218442346
0.9995742618218296
0.9995458264017248
0.9995156449742595
0.9995237722606167
0.999525483006836
0.9994286045814912
0.9994455066490165
0.9993543402870411
0.9993341477910813
0.9993450063997167
0.9992981576954219
0.9993112290868652
0.9993407670215407
0.9993558492826834
0.9994940532223178
0.9994334897561413
0.9995588933395141
0.9995953067931552
0.99960636440389

...

0.9995160617468447
0.9992798777170665
0.9993393741172465
0.9993280859784909
0.9994666760338314
0.9993433068900587
0.999396425151046
0.9993823684633347
0.9994098149809822
0.9993956504181534
0.9993134118536946
0.9993916527284089
0.9994129917981663
0.9993431020342414
0.9994013269477748
0.9993453352295908
0.9994133427112186
0.9993708907374005
0.999408608015704
0.9993196065080971
0.9994777667596852
0.9995043686968632
0.9995372138763274
0.9996069544509691
0.9996483354967747
0.9997066128073232
0.999690171217871
0.9997461844104343
0.999784869782893
0.9998349219779751
0.9998847382212648
0.9999303727148937
0.9999345017683522
0.9999492094427564
0.999937766028792
0.999951143871348
0.9999167381444648
0.9998850137592485
0.9998658459791832
0.9998674346374569
0.9998744698847435
0.9997173255611987
0.9997563426983055
0.9997473080416462
0.9997044196764335
0.9996772543574957
0.9996441267213083
0.9996176370512333
0.9996335856403006
0.9996249673937472
0.9996025204247975
0.9996432855704169
0.9996251647865346

0.9998876141575882
0.9998263389643379
0.9997784728348132
0.9997687141487648
0.9997458173560975
0.9997312226904916
0.9996180703462356
0.9996197247473121
0.9995507153677148
0.9995545813570299
0.9994754153343174
0.9993744820268954
0.9993330275714971
0.99936515901865
0.9993673855860223
0.9993256405677682
0.9993349978338916
0.9994131608107748
0.9994268833742684
0.9994946410411701
0.9995197978245507
0.999594211866768
0.9995904446229698
0.9995207758667335
0.9995822563560516
0.9995827853291096
0.9996324741435051
0.9997274610348644
0.999660561182072
0.9996693076577805
0.9995822207197496
0.9995894954046849
0.9996183534804032
0.9997263530418743
0.9997523050949254
0.9997679147429752
0.999732677000527
0.9997408240306296
0.9997422493269146
0.999757333102631
0.999739053177077
0.9997980744696633
0.9997987234550333
0.9997633638377239
0.9998039445435196
0.9997843285191814
0.9998482368427221
0.9998040351749149
0.9998020919556556
0.9998027558131598
0.9998034187441844
0.9998175846778862
0.9998395269492426


0.9999310665417238
0.9999171831412307
0.999918356135288
0.9999398371386724
0.999905080295688
0.9999036803710727
0.9998916956512313
0.9998885483748654
0.9998810450268589
0.9998808595504035
0.9997305677168798
0.9997519846996132
0.9996930065579007
0.9997034338003569
0.999665951787649
0.9996745573113459
0.9996740006383495
0.9996506067901166
0.9996427757601231
0.9996711540257405
0.9996696973348895
0.99963102502977
0.9996746712084236
0.9997158143904392
0.9996870062049039
0.999700260229719
0.9997380898257582
0.9997443031508922
0.9997547074576335
0.9997902249227225
0.9998629130077022
0.9998490441634937
0.999876983511379
0.9998529480706005
0.9999068447264778
0.9999112651660479
0.9999274091937698
0.999927042892514
0.9999199857424158
0.9999177621328311
0.9999028884474966
0.9998803055355612
0.9998999190813773
0.9998418598864721
0.9997968260232613
0.9998046927708644
0.999817338291583
0.9997804582157059
0.9997387837557353
0.9997275680541123
0.999657596264162
0.9996400048181754
0.9996520452250692
0.9

0.9998371856868623
0.9998534989656297
0.9999025957766183
0.9999354186181111
0.9999457249015131
0.9999457988398798
0.99992694369765
0.999905550779428
0.9998913743132094
0.9998776706455323
0.9998674861354608
0.9998633644829377
0.9997483653498099
0.9997265477158117
0.999729266383479
0.9996705880552402
0.9996912776537696
0.99966012764098
0.999667383120951
0.9996187312187916
0.9996508386618087
0.9996156093572441
0.9996066243748235
0.9995667469495898
0.999565258293154
0.999580520174608
0.9995863600271139
0.9995748495042064
0.9995531069140768
0.9996262718753726
0.9996354138354008
0.9996938151883319
0.9997559272105374
0.9997772245916057
0.9997948028534674
0.9997837396454041
0.9998505554382344
0.9998431982427383
0.9998634192258251
0.9998937085816165
0.9999129721999213
0.9999124216904174
0.9999150554901374
0.9999193918856876
0.9999067847713845
0.9998881200040243
0.9998526953529624
0.9998264614631258
0.9998155778626849
0.9997977660962132
0.9997570238534647
0.9997416088874383
0.9997115275012393
0.

0.9996258134525948
0.9996444002711551
0.9996405436908864
0.999636007050386
0.9997744112853519
0.9997490849288105
0.999724937697832
0.9997162368130272
0.9997735395360918
0.9997032574769854
0.9997616457518608
0.9997413087400996
0.9997943628126845
0.9998132156301783
0.9998079067873464
0.9997757083708881
0.9997966383399148
0.9998226486676206
0.999822541953979
0.9998292877634848
0.9997934926628752
0.9998383944262372
0.9998460098074744
0.9998244563194668
0.9998292111381973
0.999832766332489
0.999844262828218
0.9998200692597117
0.9998387999940511
0.9998416899531352
0.9998406074827093
0.9998394477508855
0.9998023417409814
0.999793495231392
0.9997908117060343
0.9998072941968962
0.9997944063729782
0.9997785924746626
0.9997755956009261
0.9997802760398414
0.9997356638912007
0.9997776448743672
0.9997574155817714
0.9997854956617013
0.9997565978526111
0.9996554541494636
0.9996774867016781
0.9996275011836487
0.9996486390865845
0.9996528136317046
0.9996224621973601
0.9996567731693519
0.9996394469984168

In [17]:
dis_ring = get_dis(scalc, int(10000), accep) #analitica pra todos os pontos do anel
f_denspl, f_densnl, deltaspl, deltasnl, indicesl, indices_modell = dis_ring

In [18]:
f_densp = f_denspl[:,1:]
f_densn = f_densnl[:,1:]

deltp = deltaspl[:,1:]
deltn = deltasnl[:,1:]

# Plotting the density probability function

# Testing cells

In [19]:
lname = ['BC','B1','B2','Q1','Q2','Q3','Q4','SDA1', 'QFA', 'QFB','SFP2']

all_index = el_idx_collector(acc, lname)

BC_index = np.array(all_index[0], dtype=object)
B2_index = all_index[2]
B1_index = all_index[1]
SFP2_index = all_index[-1]
SDA1_index = all_index[-4]
QFA = all_index[-3]
QFB = all_index[-2]

In [20]:
c = BC_index[1]
mplt.figure()

mplt.hist(hp[c],color='lightgrey', density=True)
mplt.hist(hn[c],color='lightgrey', density=True)

mplt.plot(deltp[c], f_densp[c], color='black')
mplt.plot(-deltn[c], f_densn[c], color='black')

mplt.xlim(-0.2,0.2)

mplt.xlabel(r'$\delta$')
mplt.ylabel('PDF normalizada')

Text(0, 0.5, 'PDF normalizada')

In [109]:
plot_hdis(acc, lindex)

# Calculating the standard deviation along the ring

In [103]:
plot_hdis(acc, lindex)

In [10]:
f_denspl = f_denspl[:,1:]
f_densnl = f_densnl[:,1:]

deltpl = deltaspl[:,1:]
deltnl = deltasnl[:,1:]

## This cell calculates the standard deviation

In [ ]:
secnd_mps, secnd_mns = calc_smc(f_denspl, deltpl, f_densnl, deltnl)
secnd_mps_hist, secnd_mns_hist = calc_hist(hp,hn)

## Plotting the comparision between the analytical and monte-carlo standard deviation

In [ ]:
mplt.figure()
mplt.plot(scalc, secnd_mps,label='analytical', color='black', )
mplt.plot(scalc, secnd_mps_hist, label='monte-carlo', color='yellow')
pyaccel.graphics.draw_lattice(acc, height=0.00005, offset=-0.00005, gca=True)

mplt.legend()
mplt.show()

In [39]:
# ltime = pyaccel.lifetime.Lifetime(acc)
# dic = ltime.accepen

# dic['accp'] = accep[1]
# dic['accn'] = accep[0]

# accp, accn = dic['accp'], dic['accn']

# npoints = int((spos[-1]-spos[0])/0.1)
# s_calc = np.linspace(spos[0], spos[-1], npoints)
# d_accp = np.interp(s_calc , spos, accp)
# d_accn = np.interp(s_calc, spos, -accn)

# h = 10
# s_calc[h], d_accn[h]

# # eu to trabalhando com um ponto s do anel nas coordenadas originais antes da interpolação

# # isso significa que eu to trabalhando com uma posição de spos

# pos_s=spos[h]
# mais_prox = s_calc-pos_s
# indix=np.argmin(np.abs(mais_prox))
# s_calc[indix], spos[h]
# # mas eu vou começar a trabalhar com uma posição de s_calc

# # para isso eu preciso descobrir qual s_calc é mais proximo possível de spos[h]

# f_ = ltime.touschek_data['touschek_coeffs']['f_int_p']

# mplt.figure()
# mplt.plot(s_calc, f_.squeeze())
# mplt.show()

In [285]:
# env = envelopes[idx]
# part1, part2= create_particles(env, num_part)
# part1_new, part2_new = scatter_particles(part1, part2)

# fact = 0.9400846503709537  # de_min = 1e-2

# phim = np.pi/2*fact
# chi, cross = get_cross_section_distribution(phim, num_part)

# acp=accep[1][idx]
# acpn=accep[0][idx]
# check1 = acp - part1_new[4]
# cond1=check1<0
# check2 = -(acpn - part2_new[4])
# cond2=check2<0
# ind1 = np.where(cond1)[0]
# ind2 = np.where(cond2)[0]

# print(part1_new[4][ind1].shape, part2_new[4][ind2].shape)

# mplt.figure()
# mplt.hist(part1_new[4][ind1], bins=200, density=True)
# mplt.hist(part2_new[4][ind2], bins=200, density=True)
# mplt.show()

In [287]:
# fact = 0.9400846503709537  # de_min = 1e-2
# phim = np.pi/2*fact
# chi, cross = get_cross_section_distribution(phim, num_part)

# phim = np.pi/2 - 1e-10
# chi1, cross1 = get_cross_section_distribution(phim, num_part)

# phim = np.pi/2 - 1e-5
# chi2, cross2 = get_cross_section_distribution(phim, num_part)

# fig, ax = mplt.subplots(1, 1)

# ax.plot(np.pi/2-chi, cross, color='blue')
# ax.plot(np.pi/2-chi1, cross1, color='black')
# ax.plot(np.pi/2-chi2, cross2, color='red')
# # ax.set_yscale('log')
# ax.set_xscale('log')
# fig.show()

In [ ]:
# display(np.diag(np.einsum('ji,ki -> jk', part1, part1)/num_part))
# display(np.diag(np.einsum('ji,ki -> jk', part1_new, part1_new)/num_part))
# display(np.diag(env))

fig, (ax, ay) = mplt.subplots(1, 2, figsize=(10, 4))

# ax.plot(part1[5], part1[4], 'o')
# ay.plot(part2[5], part2[4], 'o')
ax.hist(part1[4], bins=500, color='blue')
ay.hist(part2[4], bins=500, color='blue')

# ax.plot(part1_new[5], part1_new[4], 'o')
# ay.plot(part2_new[5], part2_new[4], 'o')
ax.hist(part1_new[4], bins=500, color='orange')
ay.hist(part2_new[4], bins=500, color='orange')

fig.tight_layout()
fig.show()